## This notebook prepares and shuts down the test-stand.

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the CWFS Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

In [ ]:
await script.atcs.enable_dome_following()

In [ ]:
await script.atcs.enable(settings={'athexapod':'ncsa', 'ataos':'current','atdome':'current'})

In [ ]:
await script.atcs.enable

In [ ]:
tmp=await salobj.set_summary_state(script.atcs.rem.athexapod, salobj.State.ENABLED, settingsToApply='ncsa',timeout=20)
print(tmp)

In [ ]:
tmp=await salobj.set_summary_state(script.atcs.rem.ataos, salobj.State.ENABLED, settingsToApply='current',timeout=20)
print(tmp)

In [ ]:
await script.atcs.rem.ataos.cmd_setLogLevel.set_start(level=logging.DEBUG)

In [ ]:
tmp=await salobj.set_summary_state(script.atcs.rem.atpneumatics, salobj.State.ENABLED,timeout=20)
print(tmp)

In [ ]:
await script.atcs.prepare_for_onsky()

In [ ]:
await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True, timeout=15)

In [ ]:
tmp = await script.atcs.rem.ataos.evt_settingVersions.aget()
print(tmp)

In [ ]:
script.atcs?

In [ ]:
await script.atcs.shutdown()

In [ ]:
await script.atcs.enable(settings={'athexapod':'ncsa', 'ataos':'current','atdome':'current'})